# TPM_rsem_tximport_DESeq2.R

### install packages if necessary
if (!requireNamespace("BiocManager", quietly = TRUE))
  install.packages("BiocManager")
BiocManager::install("tximportData")
BiocManager::install("tximport")
BiocManager::install("DESeq2")

In [ ]:
library(tximportData)
library(tximport)
library(DESeq2)

In [ ]:
samplesheet <- read.csv("/workspace/nf-rnaseq/con_vin_results/pipeline_info/samplesheet.valid.csv")
samples <- strsplit(samplesheet$sample, split = "_T1")

In [ ]:
samples

In [ ]:
files <- file.path("/workspace/nf-rnaseq/con_vin_results/star_rsem", paste0(samples, ".genes.results"))
file.exists(files)

In [ ]:
txi.rsem <- tximport(files, type = "rsem", txIn = FALSE, txOut = FALSE)

In [ ]:
txi.rsem$length[txi.rsem$length == 0] <- 1 # https://support.bioconductor.org/p/92763/
head(txi.rsem$counts)

In [ ]:
dim(txi.rsem$counts)

In [ ]:
colnames(txi.rsem$counts) <- samples
head(txi.rsem$counts)

In [ ]:
sampleTable <- data.frame(condition = c(rep("Control", 29), rep("Vinclozolin", 27)))

In [ ]:
rownames(sampleTable) <- colnames(txi.rsem$counts)

In [ ]:
dds <- DESeqDataSetFromTximport(txi.rsem, sampleTable, ~condition)
dds <- DESeq(dds)
res <- results(dds)
res

In [ ]:
summary(res)

In [ ]:
write.csv(as.data.frame(res),file="con_vin_results/star_rsem/con_vin_deseq2_diffexp.csv")